<a href="https://colab.research.google.com/github/namthuan2304/ML_2024/blob/main/Lab_6_21130556_%C4%90%E1%BA%B7ngNamThu%E1%BA%ADn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab is to deal with classification task using **Random Forests** and **Naïve Bayes** algorithms with/without **Feature Selection**.

*   **Deadline: 23:59, 15/04/2024**



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/MyDrive'

Mounted at /content/gdrive
/content/gdrive/MyDrive/MyDrive


# Import libraries

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from pandas import read_csv
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.datasets import load_iris, load_breast_cancer
from prettytable import PrettyTable
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_selection import RFE, SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB, GaussianNB, BernoulliNB, MultinomialNB, ComplementNB
from pandas import Series
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, OneHotEncoder, LabelEncoder
from pandas import get_dummies, concat

%pylab inline
#%run Utils.ipynb
# Warnings configuration
# ==============================================================================
import warnings
# warnings.filterwarnings('ignore')


Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


#Task 1.
Task 1. Compare the performance of selected classification algorithms including **Random forest**, **NaiveBayes**, and **SVM** with **FASHION** dataset based on **accuracy, precision, recall, f1** measures according to **without using selection feature** and **using selection feature**.



In [48]:
classifier_map = {
    "RandomForestClassifier": "Random forest",
    "GaussianNB": "NaiveBayes GaussianNB",
    "SVC": "SVM",
    "DecisionTreeClassifier": "Decision tree",
    "LogisticRegression": "Logistic Regression",
    "KNeighborsClassifier": "kNN k=7"
}

def evaluate_model(classifier, X_train, X_test, y_train, y_test):
    classifier.fit(X_train, y_train.values.ravel())
    y_pred = classifier.predict(X_test)
    ac = metrics.accuracy_score(y_test, y_pred)
    pre = metrics.precision_score(y_test, y_pred, average="macro")
    recall = metrics.recall_score(y_test, y_pred, average="macro")
    f1 = metrics.f1_score(y_test, y_pred, average="macro")
    algoName = classifier_map.get(classifier.__class__.__name__)

    if "SVM" in algoName:
        kernel_name = getattr(classifier, 'kernel', None)
        if kernel_name is not None:
            algoName += " " + kernel_name

    return [algoName, ac, pre, recall, f1]

In [55]:
  data_train = read_csv("fashion_train.csv")
  data_test = read_csv("fashion_test.csv")

  X_train = data_train.drop(columns=['y'])
  y_train = data_train['y']
  X_test = data_test.drop(columns=['y'])
  y_test = data_test['y']

  table = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])

  print("Without using selection feature ")
  table.add_row(evaluate_model(RandomForestClassifier(), X_train, X_test, y_train, y_test))
  table.add_row(evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test))
  table.add_row(evaluate_model(SVC(kernel='linear'), X_train, X_test, y_train, y_test))
  table.add_row(evaluate_model(SVC(kernel='rbf'), X_train, X_test, y_train, y_test))
  table.add_row(evaluate_model(SVC(kernel='sigmoid'), X_train, X_test, y_train, y_test))
  table.add_row(evaluate_model(SVC(kernel='poly'), X_train, X_test, y_train, y_test))
  print(table)

  print("Using selection feature Univariate Selection")
  table2 = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])

  selector = SelectKBest(chi2, k=750)
  Xtrain_new = selector.fit_transform(X_train, y_train)
  Xtest_new = selector.fit_transform(X_test, y_test)
  #X_train1, X_test1, y_train1, y_test1 = train_test_split(Xtrain_new, y_train, test_size=0.2)

  table2.add_row(evaluate_model(RandomForestClassifier(), Xtrain_new, Xtest_new, y_train, y_test))
  table2.add_row(evaluate_model(GaussianNB(), Xtrain_new, Xtest_new, y_train, y_test))
  table2.add_row(evaluate_model(SVC(kernel='linear'), Xtrain_new, Xtest_new, y_train, y_test))
  table2.add_row(evaluate_model(SVC(kernel='rbf'), Xtrain_new, Xtest_new, y_train, y_test))
  table2.add_row(evaluate_model(SVC(kernel='sigmoid'), Xtrain_new, Xtest_new, y_train, y_test))
  table2.add_row(evaluate_model(SVC(kernel='poly'), Xtrain_new, Xtest_new, y_train, y_test))
  print(table2)

Without using selection feature 
+-----------------------+----------+--------------------+--------------------+--------------------+
|       Classifier      | Accuracy |     Precision      |       Recall       |      F1 Score      |
+-----------------------+----------+--------------------+--------------------+--------------------+
|     Random forest     |  0.817   | 0.8149062073172104 | 0.8142043857745425 | 0.8084416677737233 |
| NaiveBayes GaussianNB |  0.556   | 0.5788628371304589 | 0.559496772854223  | 0.5256907025966638 |
|       SVM linear      |  0.809   | 0.8117729613602249 | 0.8080638899207333 | 0.8054464844513045 |
|        SVM rbf        |  0.812   | 0.8070229601657756 | 0.8098864991670668 | 0.8068213516036611 |
|      SVM sigmoid      |  0.394   | 0.4538437282083073 | 0.3964375751137075 | 0.3891539162597036 |
|        SVM poly       |  0.747   | 0.7622399866233962 | 0.7442901360474709 | 0.7457242413882829 |
+-----------------------+----------+--------------------+----------

#Task 2.
For given bank dataset (**bank.csv**) having the following attributes :
1.	**age** (numeric)
2.	**job** : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3.	**marital** : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4.	**education** (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5.	**default**: has credit in default? (categorical: 'no','yes','unknown')
6.	**housing**: has housing loan? (categorical: 'no','yes','unknown')
7.	**loan**: has personal loan? (categorical: 'no','yes','unknown')
8.	**contact**: contact communication type (categorical: 'cellular','telephone')
9.	**month**: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10.	**day_of_week**: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11.	**duration**: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
12.	**campaign**: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13.	**pdays**: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14.	**previous**: number of contacts performed before this campaign and for this client (numeric)
15.	**poutcome**: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
Output variable (desired target):
16.	**y**. has the client subscribed a term deposit? (binary: 'yes','no')



#2.1. Apply StandardScaler() function to columns that contains numerical data ('age', 'balance', 'day', 'campaign', 'pdays', 'previous')

In [119]:
bank_data = read_csv("bank.csv")
columns = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']
sc = StandardScaler()
bank_data[columns] = sc.fit_transform(bank_data[columns])
for col in columns:
    min_value = bank_data[col].min()
    if min_value < 0:
        bank_data[col] += abs(min_value)
bank_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,3.441666,admin.,married,secondary,no,2.849375,yes,no,unknown,0.475039,may,1042,0.000000,0.000000,0.000000,unknown,yes
1,3.189837,admin.,married,secondary,no,2.136876,no,no,unknown,0.475039,may,1467,0.000000,0.000000,0.000000,unknown,yes
2,1.930691,technician,married,secondary,no,2.516689,yes,no,unknown,0.475039,may,1389,0.000000,0.000000,0.000000,unknown,yes
3,3.105894,services,married,secondary,no,2.890612,yes,no,unknown,0.475039,may,579,0.000000,0.000000,0.000000,unknown,yes
4,3.021951,admin.,married,tertiary,no,2.179973,no,no,unknown,0.475039,may,673,0.367383,0.000000,0.000000,unknown,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,1.259146,blue-collar,single,primary,no,2.123234,yes,no,cellular,2.256435,apr,257,0.000000,0.000000,0.000000,unknown,no
11158,1.762804,services,married,secondary,no,2.350192,no,no,unknown,1.781396,jun,83,1.102149,0.000000,0.000000,unknown,no
11159,1.175203,technician,single,secondary,no,2.131915,no,no,cellular,2.137675,aug,156,0.367383,0.000000,0.000000,unknown,no
11160,2.098577,technician,married,secondary,no,2.122924,no,yes,cellular,0.831318,may,9,0.367383,1.590755,2.181592,failure,no


#2.2. Apply Encode Categorical Value (OneHotEncoder) to transfrom categorical data to numerical data ('job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome')

In [120]:
cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
dummies = get_dummies(bank_data[cols], drop_first=True, dtype=int)
bank_data = concat([bank_data, dummies], axis=1)
bank_data = bank_data.drop(columns=cols, axis=1)
bank_data

,age,balance,day,duration,campaign,pdays,previous,deposit,job_blue-collar,job_entrepreneur,...,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown
0,3.441666,2.849375,0.475039,1042,0.000000,0.000000,0.000000,yes,0,0,...,0,0,0,1,0,0,0,0,0,1
1,3.189837,2.136876,0.475039,1467,0.000000,0.000000,0.000000,yes,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1.930691,2.516689,0.475039,1389,0.000000,0.000000,0.000000,yes,0,0,...,0,0,0,1,0,0,0,0,0,1
3,3.105894,2.890612,0.475039,579,0.000000,0.000000,0.000000,yes,0,0,...,0,0,0,1,0,0,0,0,0,1
4,3.021951,2.179973,0.475039,673,0.367383,0.000000,0.000000,yes,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,1.259146,2.123234,2.256435,257,0.000000,0.000000,0.000000,no,1,0,...,0,0,0,0,0,0,0,0,0,1
11158,1.762804,2.350192,1.781396,83,1.102149,0.000000,0.000000,no,0,0,...,0,1,0,0,0,0,0,0,0,1
11159,1.175203,2.131915,2.137675,156,0.367383,0.000000,0.000000,no,0,0,...,0,0,0,0,0,0,0,0,0,1
11160,2.098577,2.122924,0.831318,9,0.367383,1.590755,2.181592,no,0,0,...,0,0,0,1,0,0,0,0,0,0


#2.3. Apply **Decision tree, Logistic Regression, Random forest, kNN, NaïveBayes, SVM** algorithms to the preproceed dataset in the previous steps. Then compare the obtained results using **accuracy, precision, recall, f1** measures.

In [112]:
X_train, X_test, y_train, y_test = train_test_split(bank_data.drop(columns=['deposit']), bank_data['deposit'], test_size=0.2)
table = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])
table.add_row(evaluate_model(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(LogisticRegression(max_iter=50000), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(RandomForestClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(KNeighborsClassifier(n_neighbors=7), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='linear'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='rbf'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='sigmoid'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='poly'), X_train, X_test, y_train, y_test))
print(table)

+-----------------------+--------------------+--------------------+--------------------+--------------------+
|       Classifier      |      Accuracy      |     Precision      |       Recall       |      F1 Score      |
+-----------------------+--------------------+--------------------+--------------------+--------------------+
|     Decision tree     | 0.7899686520376176 | 0.7889993086343352 | 0.7890342436609095 | 0.7890166161373902 |
|  Logistic Regression  | 0.825794894760412  | 0.8252632829202629 | 0.8244166199253429 | 0.8247793344439986 |
|     Random forest     | 0.8499776085982983 | 0.8505145498952125 | 0.8520568600145704 | 0.849869217203803  |
|        kNN k=7        | 0.7577250335871025 | 0.7572880796263974 | 0.7549388655719527 | 0.7556454449596297 |
| NaiveBayes GaussianNB | 0.7062248096730855 | 0.7295491992858754 | 0.6935582396895086 | 0.6891740720936351 |
|       SVM linear      | 0.8266905508284819 | 0.8258151019541207 |  0.82635074367058  | 0.8260452881188891 |
|        S

#2.4. Using a selection feature technique to above dataset, then compare the classification results with those in Task 2.3.

In [113]:
selector = SelectKBest(chi2, k=10)
X_new = selector.fit_transform(bank_data.drop(columns=['deposit']), bank_data['deposit'])
X_train, X_test, y_train, y_test = train_test_split(X_new, bank_data['deposit'], test_size=0.2)
table = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])
table.add_row(evaluate_model(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(LogisticRegression(max_iter=50000), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(RandomForestClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(KNeighborsClassifier(n_neighbors=7), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='linear'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='rbf'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='sigmoid'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='poly'), X_train, X_test, y_train, y_test))
print(table)

+-----------------------+--------------------+--------------------+--------------------+--------------------+
|       Classifier      |      Accuracy      |     Precision      |       Recall       |      F1 Score      |
+-----------------------+--------------------+--------------------+--------------------+--------------------+
|     Decision tree     | 0.7451858486341245 | 0.7460218252526281 | 0.744496071775365  | 0.7445431963238969 |
|  Logistic Regression  | 0.8119122257053292 | 0.8173067779380903 | 0.8106174414778792 | 0.810612302264268  |
|     Random forest     | 0.7989252127183162 | 0.798969341297356  | 0.7986867130510147 | 0.7987700817570489 |
|        kNN k=7        | 0.7787729511867443 | 0.7787033471326338 | 0.778777154504819  | 0.7787246248896735 |
| NaiveBayes GaussianNB | 0.716077026421854  | 0.7695153897867315 | 0.7116927879561195 | 0.6985110848747214 |
|       SVM linear      | 0.8213166144200627 | 0.8213667285095856 | 0.8211036746354656 | 0.8211875154158728 |
|        S

#Task 3.
For a given dataset (**mobile price classification**), perform feature selection techniques and then compare the performance of selected classification algorithms (**Decision Tree, kNN, Logistic Regression, SVM, Random Forest, and NaiveBayes**) based on **accuracy, precision, recall, and f1** measures.


In [114]:
mobile_data = read_csv("mobile.csv")
selector = SelectKBest(chi2, k=10)
X_new = selector.fit_transform(mobile_data.drop(columns=['price_range']), mobile_data['price_range'])
X_train, X_test, y_train, y_test = train_test_split(X_new, mobile_data['price_range'], test_size=0.2)
table = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])
table.add_row(evaluate_model(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(KNeighborsClassifier(n_neighbors=7), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(LogisticRegression(max_iter=50000), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='linear'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='rbf'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='sigmoid'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='poly'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(RandomForestClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test))
print(table)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


+-----------------------+----------+---------------------+--------------------+---------------------+
|       Classifier      | Accuracy |      Precision      |       Recall       |       F1 Score      |
+-----------------------+----------+---------------------+--------------------+---------------------+
|     Decision tree     |  0.8375  |  0.8384465919667355 | 0.8411641081871345 |  0.8395789506769615 |
|        kNN k=7        |  0.9175  |  0.9182624517612085 | 0.9192799707602339 |  0.9183077381957534 |
|  Logistic Regression  |   0.95   |  0.9508115468409586 | 0.9528216374269006 |  0.9515224054120801 |
|       SVM linear      |   0.98   |  0.9804391932211483 | 0.9806432748538012 |  0.9804901976904148 |
|        SVM rbf        |  0.9525  |  0.9535211138784959 | 0.9532127192982456 |  0.952419826352771  |
|      SVM sigmoid      |  0.115   | 0.07474315068493151 | 0.1267361111111111 | 0.09367433414043583 |
|        SVM poly       |  0.955   |  0.9548281967112661 | 0.9559155701754386 |  0

#Task 4.
For a given dataset (named **mushroom.csv**) with the following information:

*   Attribute Information: (**classes**: edible=e, poisonous=p)
*   **cap-shape**: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
*   **cap-surface**: fibrous=f,grooves=g,scaly=y,smooth=s
*   **cap-color**: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
*   **bruises**: bruises=t,no=f
*   **odor**: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
*   **gill-attachment**: attached=a,descending=d,free=f,notched=n
*   **gill-spacing**: close=c,crowded=w,distant=d
*   **gill-size**: broad=b,narrow=n
*   **gill-color**: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
*   **stalk-shape**: enlarging=e,tapering=t
*   **stalk-root**: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
*   **stalk-surface-above-ring**: fibrous=f,scaly=y,silky=k,smooth=s
*   **stalk-surface-below-ring**: fibrous=f,scaly=y,silky=k,smooth=s
*   **stalk-color-above-ring**: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
*   **stalk-color-below-ring**: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
*   **veil-type**: partial=p,universal=u
*   **veil-color**: brown=n,orange=o,white=w,yellow=y
*   **ring-number**: none=n,one=o,two=t
*   **ring-type**: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
*   **spore-print-color**: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
*   **population**: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
*   **habitat**: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

#Task 4.1. Apply approriate reprocessing techniques for the mushroom dataset

In [125]:
mushroom_data = read_csv("mushrooms.csv")
cols = [col for col in mushroom_data.columns if col != 'class']
dummies = get_dummies(mushroom_data[cols], drop_first=True, dtype=int)
mushroom_data = concat([mushroom_data, dummies], axis=1)
mushroom_data = mushroom_data.drop(columns=cols, axis=1)
mushroom_data

,class,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,p,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,e,0,0,0,0,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2,e,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
3,p,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,e,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
8120,e,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
8121,e,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
8122,p,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0


#Task 4.2. Apply classification algorithms (**Decision Tree, Logistic Regression, SVM, kNN, Random Forest, NaiveBayes**) to the reprocessed dataset

In [127]:
X_train, X_test, y_train, y_test = train_test_split(mushroom_data.drop(columns=['class']), mushroom_data['class'], test_size=0.2)
table = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])
table.add_row(evaluate_model(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(LogisticRegression(max_iter=50000), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='linear'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='rbf'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='sigmoid'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='poly'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(KNeighborsClassifier(n_neighbors=7), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(RandomForestClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test))
print(table)

+-----------------------+--------------------+--------------------+--------------------+--------------------+
|       Classifier      |      Accuracy      |     Precision      |       Recall       |      F1 Score      |
+-----------------------+--------------------+--------------------+--------------------+--------------------+
|     Decision tree     |        1.0         |        1.0         |        1.0         |        1.0         |
|  Logistic Regression  | 0.9993846153846154 | 0.9993975903614458 | 0.9993718592964824 | 0.999384345866536  |
|       SVM linear      |        1.0         |        1.0         |        1.0         |        1.0         |
|        SVM rbf        |        1.0         |        1.0         |        1.0         |        1.0         |
|      SVM sigmoid      | 0.9581538461538461 | 0.9586232438405401 | 0.9578865376338872 | 0.958109284347354  |
|        SVM poly       |        1.0         |        1.0         |        1.0         |        1.0         |
|        k

#Task 4.3. Apply an feature selection method to dataset obtained in Task 4.1, then apply classification algorithm to the obtained dataset. Compare the performance of classification models before and after applying the feature selection technique in term of **accuracy, precision, recall, and F1** measures

In [128]:
selector = SelectKBest(chi2, k=13)
X_new = selector.fit_transform(mushroom_data.drop(columns=['class']), mushroom_data['class'])
X_train, X_test, y_train, y_test = train_test_split(X_new, mushroom_data['class'], test_size=0.2)
table = PrettyTable(["Classifier", "Accuracy", "Precision", "Recall", "F1 Score"])
table.add_row(evaluate_model(DecisionTreeClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(LogisticRegression(max_iter=50000), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='linear'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='rbf'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='sigmoid'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(SVC(kernel='poly'), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(KNeighborsClassifier(n_neighbors=7), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(RandomForestClassifier(), X_train, X_test, y_train, y_test))
table.add_row(evaluate_model(GaussianNB(), X_train, X_test, y_train, y_test))
print(table)

+-----------------------+--------------------+--------------------+--------------------+--------------------+
|       Classifier      |      Accuracy      |     Precision      |       Recall       |      F1 Score      |
+-----------------------+--------------------+--------------------+--------------------+--------------------+
|     Decision tree     | 0.9993846153846154 | 0.9993804213135068 | 0.9993894993894994 | 0.9993845818242963 |
|  Logistic Regression  |       0.976        | 0.9762827393405741 | 0.9759147662373469 | 0.9759918173294567 |
|       SVM linear      | 0.9821538461538462 | 0.9822205117171283 | 0.9821182401827564 | 0.982151142421574  |
|        SVM rbf        | 0.9993846153846154 | 0.9993804213135068 | 0.9993894993894994 | 0.9993845818242963 |
|      SVM sigmoid      | 0.9323076923076923 | 0.9332839674944938 | 0.9321261176099886 | 0.9322460867613971 |
|        SVM poly       | 0.9993846153846154 | 0.9993804213135068 | 0.9993894993894994 | 0.9993845818242963 |
|        k

#Finally,
Save a copy in your Github. Remember renaming the notebook.